<a href="https://colab.research.google.com/github/Ryan-Riggs/myrepo/blob/master/Effective_widths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Jfh6b4ZV0kl_IWFDelP0-UXCAseJQREg6Si_qI_1N8Q&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/4gGFGlqtjNP3Gy1lhFMnKDls0TOKznUf-v3aA2M1W1VK7nWqXjCISAo

Successfully saved authorization token.


In [ ]:

x = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_waterClassification_Jones2019.py'
fls = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
flsh = 'users/eeProject/RivWidthCloudPaper:rwc_landsat.py'
fnsLandsat = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
lsFun = 'users/eeProject/RivWidthCloudPaper:functions_Landsat578/functions_landsat.py'
riverFun = 'users/eeProject/RivWidthCloudPaper:functions_river.py'
grwl_cline = ee.FeatureCollection('users/eeProject/GRWL_summaryStats')
#fc_3spc = ee.FeatureCollection("users/rriggs/east_validation_1spc_3x/eastern_merged")
fc_3spc = ee.FeatureCollection("users/rriggs/SA_sj_min_100") ##South America. 
fc_3spc = ee.FeatureCollection("users/rriggs/na_sj_using_R_min_100")
gauges = ee.FeatureCollection("users/rriggs/Gauge_points") ##USGS Gauges 
#gauges = ee.FeatureCollection("users/rriggs/South_AM_100m_stationid")
#fc_3spc = ee.FeatureCollection("users/rriggs/Africa_min_100min_3x_1spc")

In [ ]:
jrcSummary = ee.Image("JRC/GSW1_0/GlobalSurfaceWater")
occ = jrcSummary.select('occurrence')
jrcMeta = ee.Image("JRC/GSW1_0/Metadata")
nobs = jrcMeta.select('valid_obs')
change = jrcSummary.select('change_abs')
abs_change = change.abs()
waterMax = occ.gte(0)
grwl = ee.FeatureCollection("users/eeProject/grwl")
# create image with bands ranging from quantile 0-100
quantileBreaks = ee.List.sequence(0, 100, 1)
import folium
def creatQuantileImage(l, prev):
    return(ee.Image(prev).addBands(occ.gte(ee.Image.constant(l)).rename([ee.String('q').cat(ee.Number(l).format('%03d'))])))


# rename the flags so that their value won't be replaced in the next step
# when calculating the mean river mask values
def appendStringFlag(l):
    return(ee.String(l).cat('_flag'))


In [ ]:
def Mndwi(image):
    return(image.normalizedDifference(['Green', 'Swir1']).rename('mndwi'))

def Mbsrv(image):
    return(image.select(['Green']).add(image.select(['Red'])).rename('mbsrv'))

def Mbsrn(image):
    return(image.select(['Nir']).add(image.select(['Swir1'])).rename('mbsrn'))

def Ndvi(image):
    return(image.normalizedDifference(['Nir', 'Red']).rename('ndvi'))

def Awesh(image):
    return(image.expression('Blue + 2.5 * Green + (-1.5) * mbsrn + (-0.25) * Swir2', {
    'Blue': image.select(['Blue']),
    'Green': image.select(['Green']),
    'mbsrn': Mbsrn(image).select(['mbsrn']),
    'Swir2': image.select(['Swir2'])}))

def Dswe(i):
    mndwi = Mndwi(i)
    mbsrv = Mbsrv(i)
    mbsrn = Mbsrn(i)
    awesh = Awesh(i)
    swir1 = i.select(['Swir1'])
    nir = i.select(['Nir'])
    ndvi = Ndvi(i)
    blue = i.select(['Blue'])
    swir2 = i.select(['Swir2'])

    t1 = mndwi.gt(0.124)
    t2 = mbsrv.gt(mbsrn)
    t3 = awesh.gt(0)
    t4 = (mndwi.gt(-0.44)
    .And(swir1.lt(900))
    .And(nir.lt(1500))
    .And(ndvi.lt(0.7)))
    t5 = (mndwi.gt(-0.5)
    .And(blue.lt(1000))
    .And(swir1.lt(3000))
    .And(swir2.lt(1000))
    .And(nir.lt(2500)))

    t = t1.add(t2.multiply(10)).add(t3.multiply(100)).add(t4.multiply(1000)).add(t5.multiply(10000))

    noWater = (t.eq(0)
    .Or(t.eq(1))
    .Or(t.eq(10))
    .Or(t.eq(100))
    .Or(t.eq(1000)))
    hWater = (t.eq(1111)
    .Or(t.eq(10111))
    .Or(t.eq(11011))
    .Or(t.eq(11101))
    .Or(t.eq(11110))
    .Or(t.eq(11111)))
    mWater = (t.eq(111)
    .Or(t.eq(1011))
    .Or(t.eq(1101))
    .Or(t.eq(1110))
    .Or(t.eq(10011))
    .Or(t.eq(10101))
    .Or(t.eq(10110))
    .Or(t.eq(11001))
    .Or(t.eq(11010))
    .Or(t.eq(11100)))
    pWetland = t.eq(11000)
    lWater = (t.eq(11)
    .Or(t.eq(101))
    .Or(t.eq(110))
    .Or(t.eq(1001))
    .Or(t.eq(1010))
    .Or(t.eq(1100))
    .Or(t.eq(10000))
    .Or(t.eq(10001))
    .Or(t.eq(10010))
    .Or(t.eq(10100)))

    iDswe = (noWater.multiply(0)
    .add(hWater.multiply(1))
    .add(mWater.multiply(2))
    .add(pWetland.multiply(3))
    .add(lWater.multiply(4)))

    return(iDswe.rename(['dswe']))

def ClassifyWaterJones2019(img):
    dswe = Dswe(img)
    waterMask = dswe.eq(1).Or(dswe.eq(2)).rename(['waterMask'])
    return(waterMask)

def switchGeometryLine2Endpoints(f):
        f = f.set({'lineGeometry': f.geometry()})
        l = f.geometry().coordinates()
        g = ee.Geometry.MultiPoint(l, 'EPSG:4326')
        return(f.setGeometry(g))

def switchGeometryEndpoints2Line(f):
        return(f.setGeometry(f.get('lineGeometry')).set('lineGeometry', None))

def GetCenterline(clDataset, bound):
    # // filter the GRWL centerline based on area of interest
    cl = clDataset.filterBounds(bound)
    return(cl)

def ExtractChannel(image):
    # // extract the channel water bodies from the water mask, based on connectivity to the reference centerline.
    connectedToCl = (image.Not().cumulativeCost(
        source = ee.Image().toByte().paint(grwl_cline, 1).And(image), #// only use the centerline that overlaps with the water mask
        maxDistance = 4000,
        geodeticDistance = False).eq(0))

    channel = image.updateMask(connectedToCl).unmask(0).updateMask(image.gte(0)).rename(['channelMask'])
    return(channel)


def RemoveIsland(channel):
    # /* fill in island as water if the size (number of pixels) of the island is smaller than FILL_SIZE */
    fill = channel.Not().selfMask().connectedPixelCount(333).lt(333)
    river = channel.where(fill, ee.Image(1)).rename(['riverMask'])
    return(river)

def ExtractRiver(imgIn, clData, maxDist, minIslandRemoval):
    waterMask = imgIn.select(['waterMask'])
    bound = waterMask.geometry()
    cl = GetCenterline(clData, bound)
    channelMask = ExtractChannel(waterMask, cl, maxDist)
    riverMask = RemoveIsland(channelMask, minIslandRemoval)
    return(imgIn.addBands(channelMask).addBands(riverMask))

def widths(image):
  width = (image.eq(1).reduceRegions(
  collection = filt, 
  reducer= ee.Reducer.mean(),
  ))
  flags = (image.reduceRegions(
  collection= width.map(switchGeometryLine2Endpoints), 
  reducer= ee.Reducer.max(),
))
  return(flags)


def filt_lines (f):
  return f.set('geo_type', f.geometry().type())

def maximum_no_of_tasks(MaxNActive, waitingPeriod):
	"""maintain a maximum number of active tasks
	"""
	import time
	import ee
	ee.Initialize()

	time.sleep(10)
	## initialize submitting jobs
	ts = list(ee.batch.Task.list())

	NActive = 0
	for task in ts:
		if ('RUNNING' in str(task) or 'READY' in str(task)):
			NActive += 1
	## wait if the number of current active tasks reach the maximum number
	## defined in MaxNActive
	while (NActive >= MaxNActive):
		time.sleep(waitingPeriod) # if reach or over maximum no. of active tasks, wait for 2min and check again
		ts = list(ee.batch.Task.list())
		NActive = 0
		for task in ts:
			if ('RUNNING' in str(task) or 'READY' in str(task)):
				NActive += 1
	return()
def buffer_zone (f):
  return f.buffer(5000)
def  ftr_coll (f):
  return ee.FeatureCollection(f)

def distance_fun (f):
  l = f.geometry().distance(filter_gauge.geometry())
  d = f.set('distance', l)
  return(d)

In [ ]:
riverMask = ExtractChannel(waterMax)
quantileImage = ee.Image(quantileBreaks.iterate(creatQuantileImage, ee.Image())).select('^q.*').updateMask(riverMask).unmask(0)
bn = ee.List(quantileImage.bandNames()).map(appendStringFlag)
reducer = ee.Reducer.anyNonZero().forEach(bn)

In [ ]:
collection = fc_3spc.map(filt_lines)
fc_3spc = collection.filter(ee.Filter.eq('geo_type', 'LineString'))

#fc_3spc = fc_3spc.filterBounds(rc_roi)
LT5_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5',    'B7',    'B6'];
LE7_BANDS = ['B1',   'B2',    'B3',  'B4',  'B5',    'B7',    'B6_VCID_1'];
LC8_BANDS = ['B2',   'B3',    'B4',  'B5',  'B6',    'B7',    'B10'];
STD_NAMES = ['Blue', 'Green', 'Red', 'Nir', 'Swir1', 'Swir2', 'Temp'];

#// load Landsat 5,7,8 collections:
#// TODO(GHA): combine 5, 7, and 8 collections:
LT5 = ee.ImageCollection('LANDSAT/LT5_L1T_SR').select(LT5_BANDS, STD_NAMES);
LT5 = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR').select(LT5_BANDS, STD_NAMES); 
##LE7 = ee.ImageCollection('LANDSAT/LE07/C01/T1_TOA')
##.select(LE7_BANDS, STD_NAMES);
LC8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').select(LC8_BANDS, STD_NAMES);

collection = LC8.merge(LT5)

In [ ]:
p = [5464000,
5464315,
5464420,
5464500,
5465000,
5465500,
5465700,
5474500,
5482000,
5484600,
5485500,
5487500,
5488110,
5488500,
5489500,
5490500,
5490600,
5520500,
5526000,
5527000,
5527500,
5533500,
5533600,
5534050,
5534100,
5536890,
5536995,
5537000,
5537980,
5538000,
5539660,
5541000,
5543500,
5547350,
5548500,
5551000,
5551540,
5552500,
5553500,
5558300,
5560000,
5568500,
5570500,
5585500,
5586100,
5587450,
5587455,
5587500,
6041500,
6042000,
6058000,
6058502,
6066500,
6071500,
6074000,
6078200,
6090300,
6090800,
6109000,
6109500,
6115000,
6115200,
6132000,
6177000,
6185500,
6186500,
6214500,
6217500,
6218000,
6286250,
6294000,
6294500,
6294700,
6295000,
6296120,
6309000,
6327500,
6329500,
6330000,
6338490,
6339000,
6342500,
6440000,
6452000,
6453000,
6463720,
6465000,
6465500,
6467500,
6478500,
6486000,
6601200,
6610000,
6644500,
6645500,
6646800,
6650000,
6652000,
6674500,
6684500,
6686000,
6686500,
6759910,
6760500,
6764000,
6768000,
6770000,
6770200,
6770500,
6774000,
6778000,
6780000,
6785000,
6788500,
6790500,
6790501,
6793000,
6794500,
6796000,
6801000,
6805500,
6807000,
6813500,
6818000,
6857000,
6879100,
6886000,
6887500,
6888345,
6888350,
6889000,
6891000,
6891080,
6892350,
6892500,
6893000,
6895500,
6906500,
6909000,
6920500,
6922000,
6922450,
6922500,
6925000,
6926000,
6926500,
6926510,
6934500,
6935965,
7010000,
7020500,
7022000,
7032000,
7049500,
7053000,
7054000,
7055000,
7057370,
7059000,
7060500,
7061000,
7074500,
7074850,
7076750,
7077000,
7077800,
7117500,
7144550,
7145200,
7146500,
7148140,
7150500,
7152500,
7157980,
7158000,
7159100,
7159400,
7160000,
7161000,
7161450,
7163500,
7164000,
7164200,
7164500,
7165570,
7165600,
7178600,
7190500,
7191500,
7192500,
7193500,
7194500,
7228500,
7229000,
7229050,
7229100,
7229200,
7244500,
7245000,
7246500,
7249455,
7250500,
7250550,
7250551,
7258000,
7263450,
7263500,
7265450,
7288800,
7288955,
7289000,
7302000,
7332000,
7335500,
7336820,
7337000,
7341301,
7341500,
7344400,
7348500,
7350500,
7355500,
7357000,
7357501,
7358000,
7359001,
7359002,
7359500,
7364100,
7367000,
7367005,
7367630,
7374000,
7374525,
7381490,
8038500,
8041000,
8041780,
8045500,
8063000,
8066000,
8066191,
8066250,
8067000,
8088600,
8114000,
8114500,
8116650,
8117290,
8154000,
8154510,
8314500,
8329918,
8329928,
8330150,
8332000,
8332010,
8359500,
8382800,
9120500,
9163500,
9180500,
9272400,
9307000,
9315000,
9335000,
9339500,
9357000,
9379910,
9380000,
9380002,
9383000,
9404200,
9421500,
9424000,
9427520,
9429010,
9429100,
9429300,
9429490,
9429500,
9500500,
9520360,
9520700,
9522200,
11366500,
11368500,
11369000,
11369500,
11378000,
11383800,
11425000,
11425500,
11447500,
11477000,
11530500,
12150800,
12196150,
12199000,
12200500,
12301500,
12301850,
12303000,
12305000,
12306000,
12318500,
12322000,
12353000,
12369000,
12388700,
12389000,
12391000,
12391400,
12392000,
12395500,
12396500,
12398500,
12399500,
12405000,
12405500,
12417610,
12436500,
12437900,
12438000,
12450700,
12453700,
12462600,
12464500,
12472800,
12508990,
12514000,
13031500,
13057150,
13057155,
13062504,
13077000,
13081500,
13082000,
13088000,
13090355,
13134500,
13135000,
13171000,
13171620,
13172500,
13173600,
13213100,
13269000,
13290000,
13334300,
13342500,
13343000,
13343500,
13343600,
13353000,
14019200,
14103000,
14105700,
14144700,
14174000,
14191000,
14197900,
14198000,
14207700,
14211720,
14223780,
14235000,
14246900,
14322900,
14335000,
15024800,
15041200,
15056500,
15129000,
15202000,
15212000,
15214000,
15258000,
15266110,
15266150,
15281000,
15291000,
15291500,
15292000,
15292700,
15292780,
15294345,
15294350,
15294500,
15300000,
15300500,
15302000,
15302300,
15302500,
15303600,
15303900,
15304000,
15356000,
15389000,
15389100,
15389500,
15453500,
15468000,
15472000,
15478000,
15515500,
15518300,
15564800,
15564900,
15565200,
15565447,
15744000,
15744500,
15746000,
15875000,
15880000
]

In [ ]:
distFilter = ee.Filter.contains(
  leftField ='.geo',
  rightField = '.geo'
  )

ee.Filter.contains()

distSaveAll = ee.Join.saveAll(
    matchesKey ='points',
    measureKey = 'distance');


  
def ij (state):
  nPowerPlants = ee.List(state.get('points'))
  mean = ee.FeatureCollection(nPowerPlants).aggregate_mean('width_m')
  return ee.FeatureCollection(state.set({'width_m': mean, 'COMID':distinct_comid.first().get('COMID')}))


def connected (f):
  con = f.clipToBoundsAndScale(filt)
  return con

def effective_width_1 (f):
  lng = (f.eq(1).reduceRegions(
  collection= grwl_cline.filterBounds(filt), 
  reducer= ee.Reducer.count(),
  ))
  return lng 


def effective_width_2 (con):
  sum = (con.select('channelMask').eq(1).reduceRegions(
  collection= fi1.flatten().geometry(), 
  reducer= ee.Reducer.sum(),
  ))
  return sum


def Area_fun (feature):
  d = ee.Number(feature.get('sum'))
  id = feature.getString('system:index').slice(0,22)
  return feature.set({'Area': ee.Number(d).multiply(900), 'id': id})



def len_fun (feature):
  d = ee.Number(feature.get('count'))
  id = feature.getString('system:index').slice(0,22)
  return feature.set({'length': ee.Number(d).multiply(30), 'id':id})

toyFilter = ee.Filter.equals(leftField = 'id',
                             rightField ='id');


innerJoin = ee.Join.inner()


def fc_function (f):
  p = ee.Feature(f.get('primary'))
  s = ee.Feature(f.get('secondary'))
  properties = p.copyProperties(s)
  return properties


def effW_fun (feature):
  Area = ee.Number(feature.get('Area'))
  Length = ee.Number(feature.get('length'))
  return feature.set({'Effective_width': Area.divide(Length)})

def fields (f):
  return f.set({'COMID': distinct_comid.first().get('COMID'), 'width_m':intersectJoined.flatten().first().get('width_m'), 'ID': fc_id.first().get('ID_2')})


def set_id (f):
  return f.set({'id': fc_id})



In [ ]:
#################### Calculate quantile widths for each gauge/scene. 
for i in range(len(p)):
  filter_gauge = gauges.filter(ee.Filter.eq('SITE_NUM', p[i])) ##'SITE_NUM' changed to 'Sttn_Nm'
  buff1 = filter_gauge.map(buffer_zone)
  fi1 = buff1.map(ftr_coll)
  filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
  fff = filt1.map(distance_fun)

  fff = filt1.map(distance_fun)
  filt = fff.filter(ee.Filter.lte('distance', 1000))
  fc_id = fff.limit(1, 'distance')
  fc_id = fc_id.first().get('ID_2')
  spatialJoined = distSaveAll.apply(fi1.flatten(), filt, distFilter);
  distinct_comid = filt.distinct('COMID')
  intersectJoined = spatialJoined.map(ij)
  fi1 = intersectJoined

  fi1 = intersectJoined.set({'COMID': distinct_comid.first().get('COMID'), 'width_m':intersectJoined.flatten().first().get('width_m')})

  quantileImage_f = connected(quantileImage)

  Area = (quantileImage_f
  .addBands(nobs.rename(['n_valid_obs']))
  .reduceRegions(
  collection= fi1.flatten().geometry(), 
  reducer= ee.Reducer.sum(),
  scale = 30,
  crs = 'EPSG:4326',
  tileScale = 1
  ))

  length = (quantileImage_f
  .addBands(nobs.rename(['n_valid_obs']))
  .reduceRegions(
  collection= grwl_cline.filterBounds(filt), 
  reducer= ee.Reducer.count(),
  scale = 30,
  crs = 'EPSG:4326',
  tileScale = 1
  ))

  Area = Area.map(set_id)
  length = length.map(set_id)

  #Area = Area.set

  #length = set_id(length)

  #result = innerJoin.apply(Area, length, toyFilter)
  #result = result.map(fc_function)
    # export widths
  task = (ee.batch.Export.table.toDrive(
  collection = length,
  description = 'Canada' + '_' + str(p[i]),
  folder = 'Effective_width_quantiles',
  fileNamePrefix = 'length_' + '_' + str(p[i]),
  fileFormat = 'csv'
  ))
  task.start()

  # task = (ee.batch.Export.table.toDrive(
  # collection = length,
  # description = 'Canada' + '_' + str(p[i]),
  # folder = 'Effective_width_quantiles',
  # fileNamePrefix = 'length_' + '_' + str(p[i]),
  # fileFormat = 'csv'
  # ))
  # task.start()
#print(output.first())
  print('task', i, 'has started')
  maximum_no_of_tasks(3, 60)

task 0 has started
task 1 has started
task 2 has started
task 3 has started
task 4 has started
task 5 has started
task 6 has started
task 7 has started
task 8 has started
task 9 has started
task 10 has started
task 11 has started
task 12 has started
task 13 has started
task 14 has started
task 15 has started
task 16 has started
task 17 has started
task 18 has started
task 19 has started
task 20 has started
task 21 has started
task 22 has started
task 23 has started
task 24 has started
task 25 has started
task 26 has started
task 27 has started
task 28 has started
task 29 has started
task 30 has started
task 31 has started
task 32 has started
task 33 has started
task 34 has started
task 35 has started
task 36 has started
task 37 has started
task 38 has started
task 39 has started
task 40 has started
task 41 has started
task 42 has started
task 43 has started
task 44 has started
task 45 has started
task 46 has started
task 47 has started
task 48 has started
task 49 has started
task 50 ha

In [ ]:
####### Calculate widths for each gauge in list p
for i in range(len(p)):
  filter_gauge = gauges.filter(ee.Filter.eq('SITE_NUM', p[i]))
  buff1 = filter_gauge.map(buffer_zone)
  fi1 = buff1.map(ftr_coll)
  filt1 = fc_3spc.filterBounds(fi1.flatten().geometry())
  
  fff = filt1.map(distance_fun)
  filt = fff.filter(ee.Filter.lte('distance', 1000))
  fc_id = fff.limit(1, 'distance')
  spatialJoined = distSaveAll.apply(fi1.flatten(), filt, distFilter);
  distinct_comid = filt.distinct('COMID')
  intersectJoined = spatialJoined.map(ij)
  fi1 = intersectJoined
  
  #filt = fc_3spc.filterBounds(x1)
  filtered = collection.filterDate('1984-01-01', '2020-12-31').sort('system:time_start').filterMetadata('CLOUD_COVER', 'less_than', 10).filterBounds(filt)
  waterMask = filtered.map(ClassifyWaterJones2019)
  GetCenterline(grwl_cline, filt)
  riverMask = waterMask.map(ExtractChannel)
  connected_mask = riverMask.map(connected)
  eff_width1 = connected_mask.map(effective_width_1)
  eff_width2 = connected_mask.map(effective_width_2)
  area_map = eff_width2.flatten().map(Area_fun)
  len_map = eff_width1.flatten().map(len_fun)
  toyJoin = innerJoin.apply(area_map, len_map,  toyFilter)
  fc_test = toyJoin.map(fc_function)
  testing = fc_test.map(effW_fun)
  fields_vals = testing.map(fields)
  selection = fields_vals.select(['Effective_width', 'ID', 'id', 'COMID', 'width_m'])

  task = (ee.batch.Export.table.toDrive(
  collection = selection,
  description = 'widths_' + '_' + str(p[i]),
  folder = 'Effective_widths',
  fileNamePrefix = 'Gauge_' + '_' + str(p[i]),
  fileFormat = 'csv'
  ))

  task.start()
#print(output.first())
  print('task', i, 'has started')
  maximum_no_of_tasks(3, 60)

task 0 has started
task 1 has started
task 2 has started
task 3 has started
task 4 has started
task 5 has started
task 6 has started
task 7 has started
task 8 has started
task 9 has started
task 10 has started
task 11 has started
task 12 has started
task 13 has started
task 14 has started
task 15 has started
task 16 has started
task 17 has started
task 18 has started
task 19 has started
task 20 has started
task 21 has started
task 22 has started
task 23 has started
task 24 has started
task 25 has started
task 26 has started
task 27 has started
task 28 has started
task 29 has started
task 30 has started
task 31 has started
task 32 has started
task 33 has started
task 34 has started
task 35 has started
task 36 has started
task 37 has started
task 38 has started
task 39 has started
task 40 has started
task 41 has started
task 42 has started
task 43 has started
task 44 has started
task 45 has started
task 46 has started
task 47 has started
task 48 has started
task 49 has started
task 50 ha